<a href="https://colab.research.google.com/github/Samay-jain622/NEURAL-STYLE-TRANSFER/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG19, ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import preprocess_input as preprocess_vgg
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet
import cv2

# Function to compute Gram matrix
def gram_matrix(input_tensor):
    result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor)
    num_locations = tf.cast(input_shape[1] * input_shape[2], tf.float32)
    return result / num_locations

# Function to load VGG19 model and define layers
def load_vgg():
    vgg = VGG19(include_top=False, weights='imagenet')
    content_layers = ['block4_conv2']
    style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
    vgg.trainable = False
    content_outputs = [vgg.get_layer(layer).output for layer in content_layers]
    style_outputs = [vgg.get_layer(layer).output for layer in style_layers]
    model_outputs = content_outputs + style_outputs
    return Model(vgg.input, model_outputs)

# Function to load ResNet50 model and define layers
def load_resnet():
    resnet = ResNet50(include_top=False, weights='imagenet')
    content_layers = ['conv4_block6_out']
    style_layers = ['conv1_conv', 'conv2_block3_out', 'conv3_block4_out', 'conv4_block6_out', 'conv5_block3_out']
    resnet.trainable = False
    content_outputs = [resnet.get_layer(layer).output for layer in content_layers]
    style_outputs = [resnet.get_layer(layer).output for layer in style_layers]
    model_outputs = content_outputs + style_outputs
    return Model(resnet.input, model_outputs)

# Streamlit UI
def main():
    st.title('Neural Style Transfer')

    model_option = st.selectbox('Choose model for style transfer', ('VGG19', 'ResNet50'))

    content_file = st.file_uploader('Upload content image', type=['jpg', 'png', 'jpeg'])
    style_file = st.file_uploader('Upload style image', type=['jpg', 'png', 'jpeg'])
    button_tostyle = st.button('Stylize Image')

    if content_file and style_file and button_tostyle:
        content_image = cv2.imdecode(np.frombuffer(content_file.read(), np.uint8), cv2.IMREAD_COLOR)
        style_image = cv2.imdecode(np.frombuffer(style_file.read(), np.uint8), cv2.IMREAD_COLOR)

        if model_option == 'VGG19':
            # Convert BGR to RGB for VGG19
            content_image = cv2.cvtColor(content_image, cv2.COLOR_BGR2RGB)
            style_image = cv2.cvtColor(style_image, cv2.COLOR_BGR2RGB)
            preprocess = preprocess_vgg
            model = load_vgg()
            style_weight = 1e-2
            content_weight = 1e-4
        else:
            # ResNet50 expects BGR format, no conversion needed
            preprocess = preprocess_resnet
            model = load_resnet()
            style_weight = 1
            content_weight = 1e-2

        content_image = cv2.resize(content_image, (224, 224))
        style_image = cv2.resize(style_image, (224, 224))
        content_image = tf.image.convert_image_dtype(content_image, tf.float32)
        style_image = tf.image.convert_image_dtype(style_image, tf.float32)

        content_image_processed = preprocess(tf.expand_dims(content_image, axis=0))
        style_image_processed = preprocess(tf.expand_dims(style_image, axis=0))

        content_target = model(content_image_processed)[0]
        style_target = [gram_matrix(output) for output in model(style_image_processed)[1:]]

        opt = tf.keras.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)

        def loss_object(style_outputs, content_outputs, style_target, content_target):
            content_loss = tf.reduce_mean(tf.square(content_outputs - content_target))
            style_loss = tf.add_n([tf.reduce_mean(tf.square(gram_matrix(output) - target)) for output, target in zip(style_outputs, style_target)])
            total_loss = content_weight * content_loss + style_weight * style_loss
            return total_loss

        @tf.function
        def train_step(image):
            with tf.GradientTape() as tape:
                outputs = model(image)
                loss = loss_object(outputs[1:], outputs[0], style_target, content_target)
            gradient = tape.gradient(loss, image)
            opt.apply_gradients([(gradient, image)])
            image.assign(tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0))
            return loss

        Epochs = 1000
        stylized_image = tf.Variable(content_image_processed, dtype=tf.float32)

        for epoch in range(Epochs):
            loss = train_step(stylized_image)
            if epoch % 100 == 0:
                st.write(f'Epoch {epoch}, Loss: {loss.numpy()}')

        # Convert back to numpy array for display
        stylized_image_np = np.squeeze(stylized_image.numpy())
        stylized_image_np = np.clip(stylized_image_np, 0, 1)  # Clip values to valid range

        # Convert to RGB if needed (for ResNet50 output)
        if model_option == 'ResNet50':
            stylized_image_np = cv2.cvtColor((stylized_image_np * 255).astype(np.uint8), cv2.COLOR_BGR2RGB)
        else:
            stylized_image_np = (stylized_image_np * 255).astype(np.uint8)

        st.image(stylized_image_np, caption='Stylized Image', use_column_width=True)

    else:
        st.write('Upload content and style images to begin.')

if __name__ == '__main__':
    main()